# Requirement:

External library to import

In [80]:
import sys
sys.path.insert(0, './library/')
import zipfile
import os
import time
import pymongo as pym
import pandas as pd
import folium
import numpy as np
import requests
from shapely.geometry  import Polygon, LineString, asShape, mapping, Point
import math
import geopy
from shapely.geometry import Polygon, MultiPolygon, Point, mapping
from geopy.distance import vincenty,great_circle
from folium.plugins import FastMarkerCluster
from datetime import datetime

# Data:

1. gtfs file of the city. 
 ->[repository of gtfs file https://transitfeeds.com/]
2. pbf file of [openstreetmap](openstreetmap.org) extract from of the city/region of interest. ->[repository of osm extract: http://download.geofabrik.de/ -- https://mapzen.com/data/metro-extracts/]

# Url and paths  [**set it!**]
### mongodb settings

In [81]:
city = 'Budapest' # name of the city 
urlMongoDb = "mongodb://localhost:27017/"; # url of the mongodb database
client = pym.MongoClient(urlMongoDb)
gtfsDB = client['PublicTransportAnalysis']

### path of the gtfs files.

In [82]:
directoryGTFS = './gtfs/'+ city+ '/' # directory of the gtfs files.

### Settings of the date and the day for the computation of accessibility quantitites
the date must be in the interval of validity of the gtfs files, check it in the "calendar.txt" and "calendar_dates.txt" files inside the gtfs zip files.

In [84]:
from libConnections import printGtfsDate
printGtfsDate(directoryGTFS)

interval of validity of the gtfs files
gtfs.zip file
 calendar_dates.txt -> date:20170515 (first row)


In [85]:
day = "20170607" #hhhhmmdd
dayName = "wednesday" #name of the corresponding day

 ## Define url of the osrm server 

In [86]:
urlServerOsrm = 'http://localhost:5000/'; #url of the osrm server of the city

## Parameters thst define the resolution and extention of tesselletion and the maximum of the walking time

In [87]:
#grid step of the hexagonal tesselletion in kilometers
gridEdge = 0.4

#parameters of walking distance
timeWalk = 15 * 60 #seconds 
velocityWalk = 1.39 # m/s ***5km/h***
distanceS = timeWalk * velocityWalk

# Start of the computation

### Read stops, routes, trips, calendar and calendar_dates from gtfs

In [89]:
from libStopsPoints import loadGtfsFile
listOfFile = ['stops.txt','routes.txt','trips.txt', 'calendar.txt', 'calendar_dates.txt']#, 'stop_times.txt']#, 'shapes.txt']
loadGtfsFile(gtfsDB, directoryGTFS, city, listOfFile)

removing stops  of  Budapest
removing routes  of  Budapest
removing trips  of  Budapest
removing calendar  of  Budapest
removing calendar_dates  of  Budapest
gtfs.zip
stops.txt -> (7438,7438)
routes.txt -> (359,359)
trips.txt -> (203273,203273)
calendar_dates.txt -> (11590,11590)


## Fill the datatbase with the connections

#### Fill the database with the connections

In [91]:
from libConnections import readConnections
readConnections(gtfsDB, city, directoryGTFS, day, dayName)

number of file in calendar+calendar_dates: 1
in stops: 1

Checking the number of services active in the date selected:
file: gtfs.zip 	 total number of active service (in calendar.txt): Serv NOT FOUND!!
number of different service_id: 0


file: gtfs.zip 	 total number of active service (in calendar_dates.txt): 249
number of different service_id: 1 total number of active services found: 249
number of trips 203000
 gtfs.zip
reading stop_times.txt...
readed... converting to a list
converted...
inserting to DB....170, err 0, err_start 0, err_start_after 0
tot connections 757002


## remove stops with no connections and label with the pos field

In [92]:
from libStopsPoints import removingStopsNoConnections, setPosField
removingStopsNoConnections(gtfsDB, city)
setPosField(gtfsDB, city)

In [93]:
from libConnections import updateConnectionsStopName
updateConnectionsStopName(gtfsDB, city)

connections deleted 07002 pEnd 757002, totC 757002


# Tassel with exagons

## Read list of stops from file

In [96]:
from libStopsPoints import returnStopsList
stopsList = returnStopsList(gtfsDB, city)

tot stop 5346  stop error : 0


## Compute the box that include all stops
The edge of such box are enlarged by distanceS.

In [112]:
from libStopsPoints import boundingBoxStops, mapStops
bbox = boundingBoxStops(stopsList)
mapStops(bbox, stopsList)

## Tassel the box with exagons.

In [129]:
imp.reload(libHex)

<module 'libHex' from './library/libHex.py'>

In [130]:
from libHex import hexagonalGrid
hexBin, pointBin = hexagonalGrid(bbox, gridEdge, gtfsDB['stops'], distanceS, city)

In [116]:
from libHex import unionHexs
map_osm = folium.Map(location=latlon, zoom_start=9);
map_osm.choropleth(unionHexs(pointBin),  fill_color='#3288bd',fill_opacity=0.3, line_color='#3288bd',line_weight=2, line_opacity=1)
map_osm

In [120]:
import libHex
imp.reload(libHex)

<module 'libHex' from './library/libHex.py'>

In [121]:
from libHex import insertPoints
insertPoints(pointBin, city, gtfsDB)
print('total number of hexagons created : {0}'.format(gtfsDB['points'].find({'city':city}).count()))

total number of hexagons created : 7613


## Find the hex with walkingTime less than timeWalk from a stops

In [146]:
from libHex import pointsServed
pointsServed(gtfsDB, stopsList, urlServerOsrm, distanceS, timeWalk, city)

In [147]:
print("Number of hexagons: {0}".format(gtfsDB['points'].find({'served':True, 'city':city}).count()))

Number of hexagons: 6123


## Setting field "pos" for points for performance

In [155]:
imp.reload(libHex)

<module 'libHex' from './library/libHex.py'>

In [152]:
from libHex import settingHexsPos
settingHexsPos(gtfsDB, city)

In [156]:
from libHex import showHexs
showHexs(gtfsDB, city, 10)

# Adding the walking time between stops and points

In [160]:
from libStopsPoints import computeNeigh
computeNeigh(gtfsDB, urlServerOsrm, distanceS, timeWalk,  city)

# Compute quantities and observable

TimeList is the list of starting time for computing the isochrones

In [161]:
timeList = [7,10,13,16,19,22] # List of starting time for computing the isochrones
hStart = timeList[0]*3600

### List of connections

In [230]:
imp.reload(libConnections)

<module 'libConnections' from './library/libConnections.py'>

In [231]:
from libConnections import makeArrayConnections
arrayCC = makeArrayConnections(gtfsDB, hStart, city)

Num of connection 668493, 123780, 4489, 4515]


### List of list of the points and stops neighbors

In [232]:
imp.reload(libStopsPoints)

<module 'libStopsPoints' from './library/libStopsPoints.py'>

In [233]:
from libStopsPoints import listPointsStopsN 
arraySP = listPointsStopsN(gtfsDB, city)

## Compute accessibility quantities

In [244]:
import libAccessibility
import icsa
imp.reload(icsa)
imp.reload(libAccessibility)

<module 'libAccessibility' from './library/libAccessibility.py'>

In [245]:
from icsa import computeVel

computeIsochrone = False
if 'isochrones' in gtfsDB.collection_names():
    #gtfsDB['isochrones'].delete_many({'city':city})
    pass
for timeStart in timeList:
    timeStart *= 3600
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
    computeVel(city, timeStart, arrayCC, arraySP, gtfsDB, computeIsochrone, timeStart/3600 == timeList[0])

Time Isochrone Start: 7.0
Time Isochrone Start: 10.0e : 2.2, time to finish : 0.0h, 0.0 mm
Time Isochrone Start: 13.0e : 2.2, time to finish : 0.0h, 0.0 m
Time Isochrone Start: 16.0e : 2.2, time to finish : 0.0h, 0.0 m
Time Isochrone Start: 19.0e : 2.2, time to finish : 0.0h, 0.0 m
Time Isochrone Start: 22.0e : 2.2, time to finish : 0.0h, 0.0 m


## Compute averages of the accessiblity quantities computed

In [248]:
imp.reload(libStopsPoints)

<module 'libStopsPoints' from './library/libStopsPoints.py'>

In [249]:
from libStopsPoints import computeAverage
valuesToAverage = ['velocityScore']
computeAverage(valuesToAverage, gtfsDB, city)

# RESULTS

## maps

In [250]:
from libHex import reduceGeojsonInShellSubField  
field1 = 'velocityScore'
field2 = 'avg'
color = ['#993404', "#f16913", "#fdae6b", '#74c476', '#31a354', '#006d2c', "#6baed6", "#4292c6", "#2171b5", '#08519c', '#f768a1', '#dd3497', '#ae017e', '#49006a'];
shell = [0., 2., 4., 5, 6., 7, 8., 9, 10., 11, 12., 13, 15, 17.];
print ("number of hexs in total", gtfsDB['points'].find({field1:{'$exists':True}, 'city':city}).count())
res = reduceGeojsonInShellSubField(list(gtfsDB['points'].find({'city':city})), field1, field2, color, shell)
#res = showMapHexRedux(city, gtfsDB['points'], field = field, shell = shell, save=True)
res[1]

number of hexs in total 6123
shell 0.0-2.0 -> 123 hexs
shell 2.0-4.0 -> 2075 hexs
shell 4.0-5 -> 1294 hexs
shell 5-6.0 -> 1307 hexs
shell 6.0-7 -> 948 hexs
shell 7-8.0 -> 305 hexs
shell 8.0-9 -> 66 hexs
shell 9-10.0 -> 5 hexs
